In [21]:
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import scipy as sp
from scipy import stats

In [82]:
data = []
with open('data.txt' , 'r') as f:
    for line in f.readlines():
        data.append(line.strip('\n'))

In [83]:
print(data)

['## Data', 'lsat = Dict{Symbol, Any}(', '  :cumulative Frequency =>', '    [3, 9, 11, 22, 23, 24, 27, 31, 32, 40, 40, 56, 56, 59, 61, 76, 86, 115, 129,', '     210, 213, 241, 256, 336, 352, 408, 429, 602, 613, 674, 702, 1000],', '  :response =>', '    [0 0 0 0 0', '     0 0 0 0 1', '     0 0 0 1 0', '     0 0 0 1 1', '     0 0 1 0 0', '     0 0 1 0 1', '     0 0 1 1 0', '     0 0 1 1 1', '     0 1 0 0 0', '     0 1 0 0 1', '     0 1 0 1 0', '     0 1 0 1 1', '     0 1 1 0 0', '     0 1 1 0 1', '     0 1 1 1 0', '     0 1 1 1 1', '     1 0 0 0 0', '     1 0 0 0 1', '     1 0 0 1 0', '     1 0 0 1 1', '     1 0 1 0 0', '     1 0 1 0 1', '     1 0 1 1 0', '     1 0 1 1 1', '     1 1 0 0 0', '     1 1 0 0 1', '     1 1 0 1 0', '     1 1 0 1 1', '     1 1 1 0 0', '     1 1 1 0 1', '     1 1 1 1 0', '     1 1 1 1 1],', '  :N => 1000', ')']


In [87]:
choices = []
for student in data[6 : len(data) - 2]:
    choice = []
    student_split = student.split(' ')
    for c in student_split[-5 : ]:
        if '[' in c:
            choice.append(int(c[1]))
        elif ']' in c:
            choice.append(int(c[0]))
        else:
            choice.append(int(c))
    choices.append(np.array(choice))
choices = np.array(choices)   
choices   

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 1, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 1, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0],
       [1, 0, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 1, 1],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 1],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [88]:
choices.shape

(32, 5)

In [90]:
cumulative = []

cum1 = data[3].strip(' ')
cum2 = data[4].strip(' ')[:-1]
cum1 = cum1.strip('[')
cum2 = cum2.strip(']')
cum = cum1 + cum2
cum_split = cum.split(',')
for c in cum_split:
    cumulative.append(int(c))

cumulative = np.array(cumulative)
cumulative

array([   3,    9,   11,   22,   23,   24,   27,   31,   32,   40,   40,
         56,   56,   59,   61,   76,   86,  115,  129,  210,  213,  241,
        256,  336,  352,  408,  429,  602,  613,  674,  702, 1000])

In [91]:
dis_freq = []
dis_freq.append(cumulative[0])
for i in range(len(cumulative) - 1):
    dis_freq.append(cumulative[i + 1] - cumulative[i])

dis_freq = np.array(dis_freq)
dis_freq

array([  3,   6,   2,  11,   1,   1,   3,   4,   1,   8,   0,  16,   0,
         3,   2,  15,  10,  29,  14,  81,   3,  28,  15,  80,  16,  56,
        21, 173,  11,  61,  28, 298])

Rasch Model

$$\Pr(X_{ij}=1) = \frac{e^{\alpha_j - \beta\times\theta_i}}{1 + e^{\alpha_j - \beta\times\theta_i}}$$

$\beta_n$ = ability of student n

$\delta_i$ = difficulty of item i

https://towardsdatascience.com/a-bayesian-approach-to-rasch-models-item-response-theory-cc08805cbb37

In [114]:
data_matrix = []

for idx in range(len(dis_freq)):
    for i in range(dis_freq[idx]):
        data_matrix.append(choices[idx])

data_matrix  = np.array(data_matrix)

In [115]:
data_matrix

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [177]:
alpha_initial = np.random.normal(0 , 100 , size = 5)
alpha_initial -= alpha_initial.mean()

# halfFlat is a uniform distribution between 0 and infinity
beta_initial = 90
theta_initial = np.random.normal( 0 , 1 , size = 1000)


In [112]:
proposal_alpha = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
proposal_beta = lambda x: np.random.normal(x , 0.1)
proposal_theta = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))

In [178]:
print(alpha_initial[0], beta_initial, theta_initial[0])

-76.24752606670523 90 2.365766352405828


In [179]:
idx = 0
(np.exp(-alpha_initial[idx] + beta_initial * theta_initial[0]) / (1 + np.exp(-alpha_initial[idx] + beta_initial * theta_initial[0])))

1.0

In [251]:
def initialize_chain():
    alpha = np.random.normal(0 , 100 , size = 5)
    alpha -= alpha.mean()

    # halfFlat is a uniform distribution between 0 and infinity
    beta = 1
    theta = np.random.normal( 0 , 1 , size = 1000)
    return alpha, beta, theta

In [288]:
def log_likelihood(data_matrix , alpha , beta , theta):
    eps = 1e-8
    sum = 0
    for i in range(data_matrix.shape[0]):
        for j in range(data_matrix.shape[1]):

            rasch_prob = np.exp(-alpha[j] + beta * theta[i] + eps) / (1 + np.exp(-alpha[j] + beta * theta[i] + eps))
            if data_matrix[i][j] == 0:
                if rasch_prob == 1:
                    sum += 0
                else:
                    sum += np.log(1 - rasch_prob)
            else:
                if rasch_prob == 0:
                    sum += 0
                else:
                    sum += np.log(rasch_prob)
            
    return sum

In [309]:
a, b, t = initialize_chain()
log_likelihood(data_matrix , a, b, t)

-26822.360965240172

In [329]:
def mcmc_sampler(num_chains , num_steps, burn_in):
    eps = 1e-6
    alpha_samples = []
    beta_samples = []
    theta_samples = []

    for chain_num in range(num_chains):
        alpha , beta , theta = initialize_chain()

        alpha_chain = []
        beta_chain = []
        theta_chain = []

        # we assume proposal distribution to be symmetric
        for step in range(num_steps):
            
            p_alpha = np.array([sp.stats.norm(0 , 100).pdf(alpha[i]) for i in range(alpha.size)])
            p_alpha = np.product(p_alpha)

            p_beta = sp.stats.norm(0 , 100).pdf(beta)

            p_theta = np.array([sp.stats.norm(0 , 1).pdf(theta[i]) for i in range(theta.size)])
            p_theta = np.product(p_theta)


            proposal_alpha = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
            proposal_beta = lambda x: np.random.normal(x , 0.1)
            proposal_theta = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
            
            alpha_new = proposal_alpha(alpha)
            beta_new = proposal_beta(beta)
            theta_new = proposal_theta(theta)

            p_alpha_new = np.array([sp.stats.norm(0 , 100).pdf(alpha_new[i]) for i in range(alpha_new.size)])
            p_alpha_new = np.product(p_alpha_new)

            p_beta_new = sp.stats.norm(0 , 100).pdf(beta_new)

            p_theta_new = np.array([sp.stats.norm(0 , 1).pdf(theta_new[i]) for i in range(theta_new.size)])
            p_theta_new = np.product(p_theta_new)

            likelihood = log_likelihood(data_matrix , alpha , beta , theta)
            likelihood_new = log_likelihood(data_matrix , alpha_new , beta_new , theta_new)

            print('(Chain: {}, Step: {}/{})=> Old Likelihood: {:.4f}, New Likelihood: {:.4f}'.format(chain_num+1, step+1, num_steps, likelihood, likelihood_new), end='\r')
            alpha_ratio = likelihood_new - likelihood + np.log(p_alpha_new + eps) - np.log(p_alpha + eps) \
                        + np.log(p_beta_new + eps) - np.log(p_beta + eps) + np.log(p_theta_new + eps) - np.log(p_theta + eps)

            u = np.random.rand()
            if u < min(1 , alpha_ratio):
                if step > burn_in-1:
                    alpha_chain.append(alpha_new)
                    beta_chain.append(beta_new)
                    theta_chain.append(theta_new)
                
                alpha = alpha_new
                beta = beta_new
                theta = theta_new

            else:
                alpha_chain.append(alpha)
                beta_chain.append(beta)
                theta_chain.append(theta)

        alpha_samples.append(alpha_chain)
        beta_samples.append(beta_chain)
        theta_samples.append(theta_chain)
        print('\n')
    return alpha_samples , beta_samples , theta_samples


In [324]:
s_alpha, s_beta, s_theta = mcmc_sampler(4, 25, 5)

In [328]:
means = np.zeros((1, 5))
for i in range(len(s_alpha)):
    means+=np.mean(s_alpha[i], axis = 0)
print(means/len(s_alpha))

[[-2.22420446 -1.95411178  0.92334153 -3.64536679 -0.2444831 ]]
